In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
tf.__version__

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
california_housing = fetch_california_housing()
california_housing

In [ ]:
X_train0, X_test, y_train0, y_test = train_test_split(
                 california_housing["data"],
                 california_housing["target"])

In [ ]:
sc = StandardScaler()
X_train_s = sc.fit_transform(X_train0)
X_test_s = sc.transform(X_test)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(1)
])

In [ ]:
model.compile(loss="mean_squared_error",
              optimizer="sgd",
              metrics=["mean_absolute_error"])

## custom loss

Because we said to avoid using if statements as much as possible.

In [ ]:
def my_loss_ (y_true , y_pred):
    error = tf.math.abs(y_true - y_pred)
    return tf.experimental.numpy.select(condlist=[error<0.1,error<1,error>=1],
                                         choicelist=[error,error*2,error**2])

In [ ]:
y_train0

In [ ]:
model.compile(loss=my_loss_,
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [ ]:
def my_loss2 (t1,t2):
    def my_loss (y_true , y_pred):
        error = tf.math.abs(y_true - y_pred)
        return tf.experimental.numpy.select(condlist=[error<t1,error<t2,error>=t2],
                                             choicelist=[error,error*2,error**2])
        return my_loss

In [ ]:
my_loss_2 = my_loss2(0.1,1)

In [ ]:
model.compile(loss=my_loss_2,
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [ ]:
model.fit(X_train_s, y_train0, epochs=5, validation_split=0.15)

In [ ]:
model.save('reg_model_my_loss_2.keras')

Here, when calling and loading the model, we treat that internal function as the key—the one that has both the predicted and actual values.
And for the value, we consider the outermost function, the one that has hyperparameters.
But this approach is not optimal, and the numbers we need to set as hyperparameters get lost, so the correct way is inheritance.
We need the model itself to save our hyperparameters, which goes inside
* get_config

In [ ]:
model_my_loss_2 = keras.models.load_model('reg_model_my_loss_2.keras',
                       custom_objects={'my_loss': my_loss_2
                                                # my_loss2 (0.1,1)
                                                  })

In [ ]:
model_my_loss_2.predict(X_test_s)

In [ ]:
class MyLoss(keras.losses.Loss):
    def __init__(self, t1, t2, **kwargs):
        super().__init__(**kwargs)
        self.t1 = t1
        self.t2 = t2


    def call(self, y_true, y_pred):      
        error = tf.abs(y_true - y_pred)
        return tf.where(error < self.t1, error,
                        tf.where(error < self.t2, error * 2, error ** 2))

    def get_config(self):
        # تنظیمات والد رو می‌گیریم که چیزی از قلم نیفته
        config = super().get_config()
        # t1 و t2 رو اضافه می‌کنیم که موقع لود کردن مدل گم نشن
        config.update({"t1": self.t1,
                       "t2": self.t2})
        return config


In [ ]:
my_loss_class =MyLoss(0.1,1)

In [ ]:
model.compile(loss=MyLoss(0.1,1),
                  # my_loss_class,
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [ ]:
model.fit(X_train_s, y_train0, epochs=5, validation_split=0.15)

In [ ]:
model.save('reg_model_my_loss__class.keras')

In [ ]:
model_my_loss_class = keras.models.load_model('reg_model_my_loss__class.keras',
                       custom_objects={'MyLoss': MyLoss
                                                  })

In [ ]:
model_my_loss_class.predict(X_test_s)

## custom regularizers 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import regularizers

class L1RegCustom(tf.keras.Regularizer):
    def __init__(self, l1=0.01):
        self.l1 = l1

    def __call__(self, weight_matrix):
        return 0.01 * tf.reduce_sum(tf.abs(weight_matrix))

    def get_config(self):
        return {'l1': self.l1}

layer = tf.keras.layers.Dense(5, input_dim=5,
                                 kernel_initializer='ones',
                                 kernel_regularizer=L1RegCustom(l1=0.01))

tensor = tf.keras.ops.ones(shape=(5, 5))

out = layer(tensor)

print(layer.losses)

### custom Constraint

In [ ]:
class NonNegative(keras.constraints.Constraint):

 def __call__(self, w):
   return w * tf.keras.ops.cast(ops.greater_equal(w, 0.), dtype=w.dtype)

In [ ]:
keras.layers.Dense(4, kernel_constraint=NonNegative())

## custom Initializer 

In [ ]:
class ExampleRandomNormal(tf.keras.Initializer):
    def __init__(self, mean, stddev):
        self.mean = mean
        self.stddev = stddev

    def __call__(self, shape, dtype=None, **kwargs):
        return keras.random.normal(
            shape, mean=self.mean, stddev=self.stddev, dtype=dtype
        )

    def get_config(self):  # To support serialization
        return {"mean": self.mean, "stddev": self.stddev}

In [ ]:
# ساخت نمونه از Initializer با مقادیر دلخواه
custom_initializer = ExampleRandomNormal(mean=0.0, stddev=0.05)

# تعریف مدل
model = tf.keras.models.Sequential([
    layers.Dense(10, input_dim=5, kernel_initializer=custom_initializer),
    layers.Dense(1, kernel_initializer=custom_initializer)
])

### custom activation 

In [ ]:
@tf.keras.utils.register_keras_serializable()
def custom_activation(x):
    # ترکیب 70% ReLU و 30% Sigmoid
    relu_part = tf.nn.relu(x) * 0.7
    sigmoid_part = tf.nn.sigmoid(x) * 0.3
    return relu_part + sigmoid_part


In [ ]:
# استفاده در مدل
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_dim=5),
    tf.keras.layers.Activation(custom_activation),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(tf.random.normal((100, 5)), tf.random.normal((100, 1)), epochs=5, verbose=0)

In [ ]:
class CustomActivation(tf.keras.layers.Layer):
    def __init__(self, relu_weight=0.7, sigmoid_weight=0.3, **kwargs):
        super(CustomActivation, self).__init__(**kwargs)
        self.relu_weight = relu_weight
        self.sigmoid_weight = sigmoid_weight

    def call(self, inputs):
        relu_part = tf.nn.relu(inputs) * self.relu_weight
        sigmoid_part = tf.nn.sigmoid(inputs) * self.sigmoid_weight
        return relu_part + sigmoid_part

    def get_config(self):
        config = super(CustomActivation, self).get_config()
        config.update({
            'relu_weight': self.relu_weight,
            'sigmoid_weight': self.sigmoid_weight
        })
        return config



In [ ]:
# استفاده در مدل
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_dim=5),
    CustomActivation(relu_weight=0.7, sigmoid_weight=0.3),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(tf.random.normal((100, 5)), tf.random.normal((100, 1)), epochs=5, verbose=0)

## custom metrics

Loss is for the model and doesn’t have much explicit interpretation; it’s mostly used for updating the weights.
But we use a metric to understand how well we’ve performed so far.

In [ ]:
def my_metric_r2 (y_true,y_pred):
    ss_res = tf.reduce_sum((y_true-y_pred)**2)
    ss_total = tf.reduce_sum((y_true-tf.reduce_mean(y_true))**2)
    r2 = 1 - (ss_res/ss_total+1e-6)
    return r2

In [ ]:
model.compile(loss=MyLoss(0.1,1),
                  # my_loss_class,
              optimizer="sgd",
              metrics=[my_metric_r2])

In [ ]:
model.fit(X_train_s, y_train0, epochs=5, validation_split=0.15)

In [ ]:
class WeightedMAE(tf.keras.metrics.Metric):
    def __init__(self, weight=1.0, name='weighted_mae', **kwargs):
        super().__init__(name=name, **kwargs)
        self.weight = weight
        self.total = self.add_weight(name='total', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # خطای مطلق با وزن محاسبه می‌شه
        error = tf.abs(y_true - y_pred) * self.weight
        if sample_weight is not None:
            error *= sample_weight
        self.total.assign_add(tf.reduce_sum(error))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        # میانگین وزن‌دار خطا
        return self.total / self.count

    def reset_states(self):
        # ریست متغیرها
        self.total.assign(0.0)
        self.count.assign(0.0)


In [ ]:
# مدل با متریک
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_dim=5),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=[WeightedMAE(weight=1.5)])
model.fit(tf.random.normal((100, 5)), tf.random.normal((100, 1)), epochs=5, verbose=0)

In [ ]:
import tensorflow as tf

class BinaryTruePositives(tf.keras.metrics.Metric):
    def __init__(self, name='binary_true_positives', **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_variable(
            shape=(),
            initializer='zeros',
            name='true_positives'
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        # تبدیل به نوع بولین
        y_true = tf.keras.ops.cast(y_true, "bool")
        y_pred = tf.keras.ops.cast(y_pred, "bool")

        # محاسبه True Positives
        values = tf.keras.ops.logical_and(
            tf.keras.ops.equal(y_true, True), tf.keras.ops.equal(y_pred, True))
        values = tf.keras.ops.cast(values, self.dtype)
        if sample_weight is not None:
            sample_weight = tf.keras.ops.cast(sample_weight, self.dtype)
            sample_weight = tf.keras.ops.broadcast_to(
                sample_weight, tf.keras.ops.shape(values))
            values = tf.keras.ops.multiply(values, sample_weight)
        self.true_positives.assign(self.true_positives + tf.keras.ops.sum(values))

    def result(self):
        return self.true_positives

In [ ]:
# مدل با متریک
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_dim=5),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=[BinaryTruePositives()])
model.fit(tf.random.normal((100, 5)), tf.random.normal((100, 1)), epochs=5, verbose=1)

## custom layer

In [ ]:
class SimpleDense(tf.keras.layers.Layer):
    def __init__(self,unit=36):
        super(SimpleDense,self).__init__()
        self.unit = unit

    def build(self,input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value = w_init(shape=(input_shape[-1],self.unit),dtype='float32'),
                            trainable=True)

        b_init = tf.zeros_initializer()
        # ویرگول داخل شیپ صرفا برای اینه که تاپل بشه
        self.b = tf.Variable(initial_value = b_init(shape=(self.unit,),dtype='float32'),
                            trainable=True)

    def call(self, inputs):  # Defines the computation from inputs to outputs
      return tf.matmul(inputs, self.w) + self.b

In [ ]:
# Instantiates the layer.
linear_layer = SimpleDense(4)

# This will also call `build(input_shape)` and create the weights.
y = linear_layer(tf.ones((2, 2)))
y

Here, we want to use a combination of our own custom layers.
For example, we’re going to use them several times, and each time we’ll set up this sequence again.

In [ ]:
class MyLinear(tf.keras.layers.Layer):
    def __init__(self,unit=36):
        super(MyLinear,self).__init__()
        self.unit = unit

    def build(self,input_shape):
        self.w = self.add_weight(shape=(input_shape[-1],self.unit),
                                 initializer='random_normal',
                                 trainable=True)
        
        # ویرگول داخل شیپ صرفا برای اینه که تاپل بشه
        self.b = self.add_weight(shape=(self.unit,),
                                 initializer='zeros',
                                 trainable=True)

    def call(self, inputs):  # Defines the computation from inputs to outputs
      return tf.matmul(inputs, self.w) + self.b

In [ ]:
class MyBlock(tf.keras.layers.Layer):
    def __init__(self):
        super(MyBlock,self).__init__()
        self.lin1 = MyLinear(50)
        self.lin2 = MyLinear(100)
        self.lin3 = MyLinear(100)
        self.lin4 = MyLinear(100)
    def call(self,input_):
        x = self.lin1(input_)
        x = tf.nn.relu(x)
        
        x = self.lin2(x)
        x = tf.nn.relu(x)
        
        x = self.lin3(x)
        x = tf.nn.relu(x)
        
        output = self.lin4(x)
        
        '''
        به قسمت زیر توجه کن که با این حرکت می شود در مدل خودمون یک لاس جدید تعریف کنیم که به لاس قبلی اضافه شود
        از add_loss
        می توان در خود لایه استفاده کرد
        '''
        loss_ = tf.reduce_mean(output) /2
        self.add_loss(loss_)
        return output

In [ ]:
model = keras.models.Sequential([
    MyBlock(),
    MyBlock(),
    keras.layers.Dense(1)
])

In [ ]:
model.compile(loss="mean_squared_error",
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [ ]:
model.fit(X_train_s, y_train0, epochs=5, validation_split=0.15)

## Gradient Tape

In [ ]:
x = tf.Variable(6.0)

In [ ]:
with tf.GradientTape() as g:
    y = x**2

In [ ]:
dy_dx = g.gradient(y,x)

In [ ]:
dy_dx.numpy()

But we can only show it the equations once and pass the numbers through, and after that, it’s gone. That’s why we need to do something else so that it gets saved and isn’t lost.
* persistent = True

In [ ]:
wight =tf.Variable(tf.random.normal((4,3)) )
bias = tf.Variable(tf.zeros(3, dtype = tf.float32))
x = np.array([[1.,2.,3.,4.]] , dtype = np.float32)

In [ ]:
with tf.GradientTape(persistent=True) as g:
    y = tf.matmul(x,wight) + bias
    loss = tf.reduce_mean(y**2)

In [ ]:
wight_grad , bias_grad = g.gradient(loss,[wight,bias])

You should be careful — if it’s a constant, or in other words a tensor, it won’t work correctly, and it must be a variable. However, even if a variable is added to a number, it becomes a constant.
Look at the bias now...

In [ ]:
wight =tf.Variable(tf.random.normal((4,3)) )
bias = tf.Variable(tf.zeros(3, dtype = tf.float32)) + 10
x = np.array([[1.,2.,3.,4.]] , dtype = np.float32)

In [ ]:
with tf.GradientTape(persistent=True) as g:
    y = tf.matmul(x,wight) + bias
    loss = tf.reduce_mean(y**2)

In [ ]:
wight_grad , bias_grad = g.gradient(loss,[wight,bias])

In [ ]:
bias

In [ ]:
wight

In [ ]:
type(bias)

In [ ]:
type(bias_grad)

In [ ]:
type(wight_grad)

## به بیان دیگر

In [ ]:
x1 = tf.Variable(4.0)
x2 = tf.Variable(4.0,trainable =False)
x3 = tf.Variable(4.0) *1.05
x4 = tf.constant(4.0)

In [ ]:
with tf.GradientTape(persistent=True) as g:
    y = x1**1 + x2**2 + x3**3 + x4**4  

In [ ]:
dz_dx = g.gradient(y , [x1,x2,x3,x4])

In [ ]:
# چاپ گرادیان‌ها
for i, dz in enumerate(dz_dx):
    print(f"Gradient w.r.t x{i+1}: {dz}")

حالا برای اینکه از اعداد ثابت هم بتوانیم مشتق بگیریم

In [ ]:
with tf.GradientTape(watch_accessed_variables=False) as g:
    g.watch(x4)
    y = x4 **3

In [ ]:
dy_dx = g.gradient(y,x4)

In [ ]:
print(dy_dx)

ما از خود لایه ها هم می توانیم به صورت مستقیم هم استفاده کنیم 

In [ ]:
den = tf.keras.layers.Dense(5,activation='elu')
x1 = tf.constant([[1.,2.,3.,4.]])

In [ ]:
with tf.GradientTape() as g:
    y = den(x1)
    loss = tf.reduce_mean(y **2)

In [ ]:
dloss_dw = g.gradient(loss, den.trainable_variables)

In [ ]:
dloss_dw

In [ ]:
X_train0, y_train0

In [ ]:
print(X_train_s.shape, y_train0.shape)

بچ 30 تایی یعنی به صورت زیر و ایپاک هم یعنی چند بار از روی خودش بگذره دیگه

In [ ]:
15480/30

In [ ]:
batch_size = 30

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((X_train_s, y_train0))

In [ ]:
train_data_ = train_data.shuffle(buffer_size=len(y_train0)).batch(batch_size)

تبدیل شده به همین تعداد بچ های شافل شده که در بالا نشان دادم

In [ ]:
for i , (x, y) in enumerate(train_data_):
  print(i+1,x.shape)

## training loop

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(1)
])

In [ ]:
loss_fn = tf.keras.losses.MeanSquaredError()

In [ ]:
optimazer_ = tf.keras.optimizers.SGD(learning_rate=0.01)

In [ ]:
train_mae = tf.keras.metrics.MeanAbsoluteError()

In [ ]:
epochs =10 

In [ ]:
for epoch in range(epochs):
    # Now the next for loop is for the batches
    for bach_i, (x_train, y_train) in enumerate(train_data_):
        with tf.GradientTape() as g:
            model_out = model(x_train, training=True)
            loss_val = loss_fn(y_train, model_out)
            
        dloss_dmodel = g.gradient(loss_val, model.trainable_weights)
        
        # Now we have obtained the derivatives, and here is where the weights should be updated
        optimazer_.apply_gradients(zip(dloss_dmodel, model.trainable_weights))

        # Now we want our metric to be updated each time it runs
        train_mae.update_state(y_train, model_out)
        
        if bach_i % 100 == 0:
            print('training loss', float(loss_val))
            
    # These metrics are stored all at once; with this operation, we display them at each epoch
    train_mae_val = train_mae.result()
    print('train mae', float(train_mae_val))
    # Here, since we don’t want previous epochs to be involved and want each epoch
    # to have its own metric calculation separately, we reset it
    train_mae.reset_state()


## قابلیت های tfdata

In [ ]:
import tensorflow as tf

dataset = tf.data.Dataset.range(100)
dataset = dataset.filter(lambda x: x < 5)
result = list(dataset.as_numpy_iterator())
print(result)

## tf.data capabilities
Storing data in a binary and compact form for tf.data

In [ ]:
with tf.io.TFRecordWriter('data_test_recored') as t:
    t.write(b'deep learning')
    t.write(b'python')
    t.write(b'test')

In [ ]:
file_ = ['data_test_recored']

In [ ]:
my_data_test  = tf.data.TFRecordDataset(file_)

In [ ]:
for i in my_data_test:
    print(i)

tensorflow teranform